<a href="https://colab.research.google.com/github/usamabaig1/twitter_sentiment_reach/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(123)
import re as re


In [15]:
#installing tweet-preprocessor
!pip install tweet-preprocessor

In [85]:
tweetsdf  = pd.read_csv('tweets.csv', encoding='latin-1')
len(tweetsdf)

9184

# Data Cleaning


In [34]:
tweetsdf['hashtag'] = tweetsdf['content'].apply(lambda x: re.findall(r'#(\w+)', x))
# print(hashtags[0:5])
# len(hashtags)

In [67]:
import preprocessor as p

# removal of URLs, Mentions
#forming a separate feature for cleaned tweets
for i,v in enumerate(tweetsdf['content']):
  tweetsdf.loc[i,'text'] = p.clean(v)

In [87]:
#removing null values
tweetsdf = tweetsdf[-(tweetsdf['text'] == "b' '")]

In [39]:
# tweetsdf[tweetsdf.duplicated(subset = ["text", 'username'])]

In [68]:
#Duplicate tweets from same user are removed from the dataset to eliminate bias
#Duplicate tweets from different users are left in the dataset
tweetsdf.drop_duplicates(subset =["text", 'username'], 
                     keep = False, inplace = True) 
len(tweetsdf)
# tweetsdf.head()

9371

# Encryption


In [88]:
!pip install cryptography

In [89]:
from cryptography.fernet import Fernet

def generate_key():
    """
    Generates a key and save it into a file
    """
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

def load_key():
    """
    Load the previously generated key
    """
    return open("secret.key", "rb").read()

def encrypt_message(message):
    """
    Encrypts a message
    """
    key = load_key()
    encoded_message = message.encode()
    f = Fernet(key)
    encrypted_message = f.encrypt(encoded_message)
    return encrypted_message


generate_key()
tweetsdf['url'] = tweetsdf['url'].apply(lambda x: encrypt_message(x))
tweetsdf['username'] = tweetsdf['username'].apply(lambda x: encrypt_message(x))
tweetsdf['user'] = tweetsdf['user'].apply(lambda x: encrypt_message(x))
tweetsdf['conversationId'] = tweetsdf['conversationId'].apply(lambda x: str(x))
tweetsdf['conversationId'] = tweetsdf['conversationId'].apply(lambda x: encrypt_message(x))

In [90]:

def load_key():
    """
    Load the previously generated key
    """
    return open("secret.key", "rb").read()

def decrypt_message(encrypted_message):
    """
    Decrypts an encrypted message
    """
    key = load_key()
    f = Fernet(key)
    decrypted_message = f.decrypt(encrypted_message)
    return decrypted_message.decode()

# decrypt_message(b'gAAAAABesCUIAcM8M-_Ik_-I1-JD0AzLZU8A8-AJITYCp9Mc33JaHMnYmRedtwC8LLcYk9zpTqYSaDaqFUgfz-tcHZ2TQjAgKKnIWJ2ae9GDoea6tw8XeJ4=')

# Export Data

In [91]:
tweetsdf.to_csv('Encrypted_Cleaned.csv')

In [93]:
tweetsdf[['date', 'text', 'replyCount', 'retweetCount ', 'likeCount']].to_csv('Filtered_data.csv')